Import and prepare

In [50]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import lxml.html as lh
import urllib.request
import bs4 as bs

In [51]:
#import wiki page
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Craw table and filter

In [59]:
def scrape_table_bs4(cname,cols):
    page  = urllib.request.urlopen(url).read()
    soup  = bs.BeautifulSoup(page,'lxml')
    table = soup.find("table",class_=cname)
    header = [head.findAll(text=True)[0].strip() for head in table.find_all("th")]
    data   = [[td.findAll(text=True)[0].strip() for td in tr.find_all("td")]
              for tr in table.find_all("tr")]
    data    = [row for row in data if len(row) == cols]
    raw = pd.DataFrame(data,columns=header)
    return raw
# get raw table
rawT = scrape_table_bs4("wikitable",3)

In [63]:
# filter not assigned Borough
Toronto=rawT[~rawT['Borough'].isin(['Not assigned'])]

# sort and set index
Toronto=Toronto.sort_values(by=['Postcode','Borough','Neighbourhood'], ascending=[1,1,1]).reset_index(drop=True)

# filter not assigned Neighbourhood
Toronto.loc[Toronto['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = Toronto['Borough']

# intergrate neighbourhood
Toronto = Toronto.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [65]:
#print the number of rows 
Toronto.shape

(103, 3)